# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X)


[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)#fit transform for training dataset and transform for testing set
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [12]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [16]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [17]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 3s 2ms/step - loss: 0.5739 - accuracy: 0.7329
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4605 - accuracy: 0.8051
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4320 - accuracy: 0.8108
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4207 - accuracy: 0.8159
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4112 - accuracy: 0.8210
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3961 - accuracy: 0.8339
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3798 - accuracy: 0.8439
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3673 - accuracy: 0.8516
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3592 - accuracy: 0.8556
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3539 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

In [26]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 70, 3, 0000, 4, 1, 0, 100000]])) > 0.5)
#Format expected by the 2d array is always a 2d array
#person form france , 600 credit score, gender ,age,tenure , balance , no of products,has a card , active member, salary
#1, 0, 0, 600, 1, 70, 3, 0000, 4, 1, 0, 100000

1/1 [==============================] - 0s 41ms/step
[[ True]]


### Predicting the Test set results

In [27]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 6ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)
#Confusion matrix has {{false positive , true negative},{false negative , true positive}}

[[1510   85]
 [ 197  208]]


0.859